<a href="https://colab.research.google.com/github/SANTHoshN302/AI-Lab302/blob/main/Lab9_fol_to_cnf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import re
import itertools



def remove_implications(expr):

    expr = re.sub(r'\(([^()]*)->([^()]*)\)', r'(¬\1∨\2)', expr)
    expr = expr.replace('->', '∨')
    return expr

def move_negations(expr):
    expr = expr.replace('¬(¬', '(')
    expr = expr.replace('¬(∀', '∃¬')
    expr = expr.replace('¬(∃', '∀¬')
    expr = expr.replace('¬(A∧B)', '(¬A∨¬B)')
    expr = expr.replace('¬(A∨B)', '(¬A∧¬B)')
    return expr

def standardize_variables(expr):
    vars_seen = {}
    counter = itertools.count(1)
    def repl(match):
        var = match.group(1)
        if var not in vars_seen:
            vars_seen[var] = f"x{next(counter)}"
        return vars_seen[var]
    return re.sub(r'\b([a-z])\b', repl, expr)

def skolemize(expr):

    return re.sub(r'∃[a-z]\.', '', expr)

def drop_universal(expr):

    return re.sub(r'∀[a-z]\.', '', expr)

def distribute(expr):
    changed = True
    while changed:
        new_expr = re.sub(r'\(([^()]*)∨\(([^()]*)∧([^()]*)\)\)',
                          r'((\1∨\2)∧(\1∨\3))', expr)
        new_expr = re.sub(r'\(\(([^()]*)∧([^()]*)\)∨([^()]*)\)',
                          r'((\1∨\3)∧(\2∨\3))', new_expr)
        changed = new_expr != expr
        expr = new_expr
    return expr

def fol_to_cnf(expr):
    expr = remove_implications(expr)
    expr = move_negations(expr)
    expr = standardize_variables(expr)
    expr = skolemize(expr)
    expr = drop_universal(expr)
    expr = distribute(expr)
    return expr



if __name__ == "__main__":
    # print("=== FOL to CNF Converter ===")
    # print("Example syntax: ∀x.(P(x) -> ∃y.Q(y))")
    formula = input("Enter your FOL formula: ")

    cnf = fol_to_cnf(formula)
    print("\n CNF form:", cnf)



Enter your FOL formula: ∀x.(P(x) -> ∃y.Q(y))

 CNF form: ∀x1.(P(x1) ∨ ∃x2.Q(x2))
